In [ ]:
from os.path import expanduser, join
import numpy as np
import matplotlib.pyplot as plt

from black_box_tools.db_utils import DBUtils
from black_box_tools.data_utils import DataUtils, Filters
from black_box_tools.plot_utils import PlotUtils

# Parameters

The following parameters should be set appropriately so that the correct data are read.

In [ ]:
log_db_data_name = 'data'
number_of_smart_wheels = 4

# Database restoring

Uncomment the lines below for restoring multiple MongoDB databases.

In [ ]:
# data_dirs = ['/path/to/db/dump/1', '/path/to/db/dump/2']
# subdb_names = ['db1', 'db2']
# DBUtils.restore_subdbs(db_name=log_db_data_name,
#                        data_dirs=data_dirs,
#                        subdb_names=subdb_names)

We now read the metadata of the sub-databases so that we can generate appropriate multi-database plots later.

In [ ]:
subdb_metadata = DBUtils.get_subdb_metadata(log_db_data_name)

# Data parsing

We first read the data of interest - in this case the velocity commands.

In [ ]:
sw_data_docs = DBUtils.get_subdb_docs(log_db_data_name, 'ros_sw_ethercat_parser_data', subdb_metadata)

timestamps = {}
wheel_encoder_pivots = {}
for subdb in sw_data_docs:
    timestamps[subdb] = DataUtils.get_all_measurements(sw_data_docs[subdb], 'timestamp')
    wheel_encoder_pivots[subdb] = DataUtils.get_all_measurements(sw_data_docs[subdb],
                                                                'sensors/*/encoder_pivot',
                                                                 number_of_smart_wheels,
                                                                 data_filter=Filters.MEDIAN)

# Data plotting

We plot the pivot encoder values of the individual wheels in separate subplots.

In [ ]:
fig = plt.figure(figsize=(20,5))

last_index = 0
for subdb in sw_data_docs:
    # we skip the current sub-database in case there
    # aren't any pivot encoder measurements
    if wheel_encoder_pivots[subdb].shape[0] == 0:
        continue

    subdb_indices = np.linspace(last_index, last_index+len(timestamps[subdb]), len(timestamps[subdb]))
    PlotUtils.subplot_data(fig=fig, subplot_params=(2,2,1),
                           timestamps=subdb_indices,
                           data=wheel_encoder_pivots[subdb][:,0],
                           x_label='t',
                           y_label='$rad$',
                           data_label=subdb)

    PlotUtils.subplot_data(fig=fig, subplot_params=(2,2,2),
                           timestamps=subdb_indices,
                           data=wheel_encoder_pivots[subdb][:,1],
                           x_label='t',
                           y_label='$rad$',
                           data_label=subdb)

    PlotUtils.subplot_data(fig=fig, subplot_params=(2,2,3),
                           timestamps=subdb_indices,
                           data=wheel_encoder_pivots[subdb][:,2],
                           x_label='t',
                           y_label='$rad$',
                           data_label=subdb)

    PlotUtils.subplot_data(fig=fig, subplot_params=(2,2,4),
                           timestamps=subdb_indices,
                           data=wheel_encoder_pivots[subdb][:,3],
                           x_label='t',
                           y_label='$rad$',
                           data_label=subdb)

    last_index += len(timestamps[subdb])

plt.show()